In [6]:
!pip install nltk
!pip install langdetect
!pip install iso639
!pip install PyPDF2
!pip install fpdf
!pip install Pyarrow 


   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.8 MB 435.7 kB/s eta 0:00:57
   ---------------------------------------- 0.1/24.8 MB 1.0 MB/s eta 0:00:25
    --------------------------------------- 0.6/24.8 MB 3.5 MB/s eta 0:00:08
   --- ------------------------------------ 2.0/24.8 MB 9.9 MB/s eta 0:00:03
   ------ --------------------------------- 3.9/24.8 MB 15.5 MB/s eta 0:00:02
   -------- ------------------------------- 5.3/24.8 MB 17.7 MB/s eta 0:00:02
   -------- ------------------------------- 5.4/24.8 MB 15.5 MB/s eta 0:00:02
   --------- ------------------------------ 5.6/24.8 MB 14.3 MB/s eta 0:00:02
   --------- ------------------------------ 6.0/24.8 MB 13.7 MB/s eta 0:00:02
   ---------- ----------------------------- 6.3/24.8 MB 13.4 MB/s eta 0:00:02
   ---------

In [4]:
import nltk
nltk.download('stopwords', download_dir=r'dados\nltk_data')

[nltk_data] Downloading package stopwords to dados\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer, SnowballStemmer
from langdetect import detect
import iso639
import unicodedata

nltk.data.path.append(r".\dados\nltk_data")

def process_string(string):
    """
    Função que realiza pré-processamento em um texto.

    Args:
        string (str): O texto a ser pré-processado.
        idioma (str, optional): O idioma da stopword list e do stemmer. Padrão é 'portuguese'.

    Returns:
        str: O texto pré-processado.
    """
    # Converte para minúsculas
    string = string.lower()

    # Prepara o regex para remover pontuações
    punct_regex = re.compile('[^\w\s]')

    # Remove pontuações
    string = re.sub(punct_regex, '', string)

    #Remove acentos
    string = unicodedata.normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')

    #Prepara regex para remover números
    num_regex = re.compile('\d+')

    # Remove números
    string = re.sub(num_regex, '', string)

    # Detecta o idioma do texto
    idioma = (iso639.to_name(detect(string))).lower()

    print('Idioma detectado:', idioma)

    # Define as stopwords de acordo com o idioma
    # Portugues
    if idioma == 'portuguese':
        stop_words = set(stopwords.words('portuguese'))
        stemmer = PorterStemmer()
    # Inglês
    elif idioma == 'english':
        stop_words = set(stopwords.words('english'))
        stemmer = SnowballStemmer('english')
    # Outros idiomas
    else:
        try:
            stop_words = set(stopwords.words(idioma))
            stemmer = SnowballStemmer(idioma)
        except:
            stop_words = set(stopwords.words('portuguese'))
            stemmer = PorterStemmer()
            # raise ValueError(f"Identificado idioma:{idioma}. Idioma deve ser 'portuguese' ou 'english'.")
    
    # Remove stopwords    
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Aplica stemming
    string = ' '.join([stemmer.stem(word) for word in string.split()])

    #Remove palavras com menos de 3 caracteres
    # string = ' '.join([word for word in string.split() if len(word) > 3])
    string = [word for word in string.split() if len(word) > 3]
    
    return string

C:\Users\Administrador\AppData\Local\Temp\ipykernel_25092\3390913963.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
#Carregar paginas do pdf e salvar as strings
import PyPDF2

def extrair_texto_pdf(nome_arquivo, num_max_tokens=1000):
    with open(nome_arquivo, 'rb') as arquivo:
        leitor = PyPDF2.PdfReader(arquivo)
        numero_paginas = len(leitor.pages)
        print('Numero de paginas: ', numero_paginas)
        texto = {}
        for pagina in range(numero_paginas):
            print('Extraindo Página: ', pagina)
            pagina_atual = leitor.pages[pagina]
            texto_extraido = pagina_atual.extract_text()
            texto_extraido = process_string(texto_extraido)
            tokens = [' '.join(texto_extraido[i:i+num_max_tokens]) for i in range(0, len(texto_extraido), num_max_tokens-10)]
            texto[pagina] = tokens
        print('Numero de páginas extraidas: ', len(texto))
        return texto

nome_arquivo = 'infraestrutura.pdf' # Substitua pelo nome do seu arquivo PDF
texto_extraido = extrair_texto_pdf(nome_arquivo)



Numero de paginas:  175
Extraindo Página:  0
Idioma detectado: portuguese
Extraindo Página:  1
Idioma detectado: portuguese
Extraindo Página:  2
Idioma detectado: portuguese
Extraindo Página:  3
Idioma detectado: portuguese
Extraindo Página:  4
Idioma detectado: portuguese
Extraindo Página:  5
Idioma detectado: portuguese
Extraindo Página:  6
Idioma detectado: portuguese
Extraindo Página:  7
Idioma detectado: portuguese
Extraindo Página:  8
Idioma detectado: portuguese
Extraindo Página:  9
Idioma detectado: portuguese
Extraindo Página:  10
Idioma detectado: portuguese
Extraindo Página:  11
Idioma detectado: portuguese
Extraindo Página:  12
Idioma detectado: portuguese
Extraindo Página:  13
Idioma detectado: portuguese
Extraindo Página:  14
Idioma detectado: portuguese
Extraindo Página:  15
Idioma detectado: portuguese
Extraindo Página:  16
Idioma detectado: portuguese
Extraindo Página:  17
Idioma detectado: portuguese
Extraindo Página:  18
Idioma detectado: portuguese
Extraindo Página:

In [8]:
from secret import apikey
from tenacity import retry, stop_after_attempt
import openai

openai.api_key = apikey
@retry(stop=stop_after_attempt(4))
def get_resume(text, *kwargs):
   
    prompt = f"Resuma o texto abaixo de forma bem didática e apenas com os principais pontos para entendimento: \n{text}"
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    max_tokens=350,
    temperature=0.5,
    messages=[
        {"role": "user", "content": prompt}
    ]
    )
    return completion.choices[0].message['content']

In [9]:
#start = time.time()
r = []
for pagina in texto_extraido:
    print('Resumindo página: ', pagina)
    for token in texto_extraido[pagina]:
        r.append(get_resume(token))
#end = time.time()
#print(f"Tempo de execução da função get_resume: {end - start:.2f}s")

Resumindo página:  0
Resumindo página:  1
Resumindo página:  2
Resumindo página:  3
Resumindo página:  4
Resumindo página:  5
Resumindo página:  6
Resumindo página:  7
Resumindo página:  8
Resumindo página:  9
Resumindo página:  10


In [6]:
r

['O texto fala sobre o livro "Iracema", escrito por José de Alencar. O livro retrata a história de um filho ausente que retorna à sua terra natal, onde encontra um lugar pitoresco chamado Várzea Doce. O autor descreve a natureza e a influência poderosa do ambiente tropical, comparando-a a um raio de fogo sobre a areia. O livro também menciona a brincadeira de crianças e a figura de uma boiada. Em seguida, o texto descreve a esposa do protagonista preparando um creme de buriti para refrescar o marido, que está descansando em uma rede. O livro é mencionado como uma distração para o protagonista, que talvez esteja ocupado com assuntos graves. O autor ressalta a importância das memórias da infância e descreve a inspiração patriótica do livro. O texto termina mencionando a imagem de uma mãe sorrindo para seu filho enquanto lê o livro em uma varanda com sombra.',
 'O texto fala sobre a hospitalidade e acolhimento dos cearenses, destacando a importância de receber bem os estrangeiros. Também 

In [ ]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

for pagina in r:
    pdf.multi_cell(0, 10, txt=pagina, align="L")
    pdf.ln()  # Adiciona uma nova linha após cada chamada ao MultiCell

pdf.output("infraestrutura_resumido.pdf")


''